# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import urllib.request
from azureml.core.model import Model


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-automl-experiment'

experiment=Experiment(workspace=ws, name=experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-189567
aml-quickstarts-189567
southcentralus
9b72f9e6-56c5-4c16-991b-19c652994860


# Create or Attach an AmlCompute cluster

In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-com-clst" 

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


# Data

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key="Maternal_Health_Risk_Data_Set" 
description_text = "UCI machine Learning Maternal Health Risk Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
#https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
count,1014.000000,1014.000000,1014.000000,1014.000000,1011.000000,1014.000000
mean,29.871795,113.198225,76.460552,8.725986,98.665678,74.301775
std,13.474386,18.403913,13.885796,3.293532,1.373367,8.088702
min,10.000000,70.000000,49.000000,6.000000,98.000000,7.000000
25%,19.000000,100.000000,65.000000,6.900000,98.000000,70.000000
50%,26.000000,120.000000,80.000000,7.500000,98.000000,76.000000
75%,39.000000,120.000000,90.000000,8.000000,98.000000,80.000000
max,70.000000,160.000000,100.000000,19.000000,103.000000,90.000000


# Review the Dataset Result

In [5]:
dataset.take(5).to_pandas_dataframe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98,86,high risk
1,35,140,90,13.0,98,70,high risk
2,29,90,70,8.0,100,80,high risk
3,30,140,85,7.0,98,70,high risk
4,35,120,60,6.1,98,76,low risk


In [6]:
from sklearn.model_selection import train_test_split
TRAIN_d, TEST_d = train_test_split(df, test_size=0.2, random_state=24, shuffle=True)
TRAIN_d.to_csv('train.csv',index = False)

ds_new = ws.get_default_datastore()
ds_new.upload_files(files = ['./train.csv'])
#ds_new.upload(src_dir='.',target_path='.')


train_data = Dataset.Tabular.from_delimited_files(path=[(ds_new,('train.csv'))])


"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

# Train
#### This creates a general AutoML settings object. Udacity notes: These inputs must match what was used when training in the portal. label_column_name has to be "RiskLevel" for example.

In [7]:
# Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "n_cross_validations":3,
    "primary_metric" : 'accuracy'
}

#  Put your automl config here

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="RiskLevel",   
                             #path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )

In [8]:
# Submit your experiment
#remote_run = experiment.submit(config= automl_config, show_output= True)
remote_run = experiment.submit(config= automl_config)

Submitting remote run.
No run_configuration provided, running on automl-com-clst with default configuration
Running on remote compute: automl-com-clst


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl-experiment,AutoML_07214c7f-99c5-4c12-ad2c-b4e4c0414795,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
      

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl-experiment,AutoML_07214c7f-99c5-4c12-ad2c-b4e4c0414795,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+------------------------------+------------------------------+------------------------------+
|Column name          

{'runId': 'AutoML_07214c7f-99c5-4c12-ad2c-b4e4c0414795',
 'target': 'automl-com-clst',
 'status': 'Completed',
 'startTimeUtc': '2022-03-21T16:52:07.384662Z',
 'endTimeUtc': '2022-03-21T17:23:05.305384Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'automl-com-clst',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-automl-experiment","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-189567","workspace_name":"quick-starts-ws-189567","region":"southcentralus","compute_targ

In [ ]:
remote_run.get_status()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
automl_best_run, automl_fitted_model = remote_run.get_output()
print("===== AutoMl Best Run ========")
print(automl_best_run)
print("\n  ===== AutoMl Best Fitted Model ========")
print(automl_fitted_model)
automl_best_run.get_tags()
automl_best_run_metrics = automl_best_run.get_metrics()

for metric_name in automl_best_run_metrics:
    metric = automl_best_run_metrics[metric_name]
    print("\n",metric_name, metric)
    


===== AutoMl Best Run ========
Run(Experiment: capstone-automl-experiment,
Id: AutoML_07214c7f-99c5-4c12-ad2c-b4e4c0414795_38,
Type: azureml.scriptrun,
Status: Completed)

  ===== AutoMl Best Fitted Model ========
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.3684210526315789, reg_lambda=0, subsample=1))], verbose=False))], flatten_transform=None, weights=[0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.0909090909

In [11]:
print(automl_fitted_model.steps)

[('datatransformer', DataTransformer(
    task='classification',
    is_onnx_compatible=True,
    enable_feature_sweeping=False,
    enable_dnn=False,
    force_text_dnn=False,
    feature_sweeping_timeout=86400,
    featurization_config=None,
    is_cross_validation=True,
    feature_sweeping_config={}
)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('0', Pipeline(
        memory=None,
        steps=[('maxabsscaler', MaxAbsScaler(
            copy=True
        )), ('lightgbmclassifier', LightGBMClassifier(
            min_data_in_leaf=20,
            random_state=None,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            )
        ))],
        verbose=False
    )), ('18', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerWrapper(
            copy=True,
            with_mean=False,
            with_std=False
        )), ('xg

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python
####  Workflow for deploying a model 
The workflow is similar no matter where you deploy your model:

<li>Register the model.
<li>Prepare an entry script.
<li>Prepare an inference configuration.
<li>Deploy the model locally to ensure everything works.
<li>Choose a compute target.
<li>Deploy the model to the cloud.
<li>Test the resulting web service.

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

##### Register Model

In [12]:
from azureml.core import Model

# Register model
automl_model_registered = remote_run.register_model(model_name='Maternal_Health_Risk_AutoML_model') 
automl_model_registered.download(target_dir="outputs", exist_ok=True)

'outputs/model.pkl'

####  List the registered model

In [13]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Maternal_Health_Risk_AutoML_model version: 1




#### Prepare an inference configuration.

In [14]:
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment
myenv = Environment.from_conda_specification(name="env", file_path="project_environment.yml")

#### Prepare an inference configuration using an entry script.

In [15]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script="scoreautoml.py",environment=myenv)


#### Deploy the model locally to ensure everything works. Will skip this part

TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 4, 
                                                       enable_app_insights=True, 
                                                       description='Maternal Health Risk AutoML model' )
model = Model(ws,'Maternal_Health_Risk_AutoML_model')

In [17]:

service=Model.deploy(workspace=ws,
                    name="maternal-health-risk-dep-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-21 17:29:04+00:00 Creating Container Registry if not exists..
2022-03-21 17:39:05+00:00 Registering the environment.
2022-03-21 17:39:06+00:00 Building image..
2022-03-21 17:49:49+00:00 Generating deployment configuration.
2022-03-21 17:49:50+00:00 Submitting deployment to compute..
2022-03-21 17:49:54+00:00 Checking the status of deployment maternal-health-risk-dep-service..
2022-03-21 17:52:32+00:00 Checking the status of inference endpoint maternal-health-risk-dep-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
print(service.scoring_uri)

http://a2b21d48-478e-463e-b747-72c9f3b14d27.southcentralus.azurecontainer.io/score


In [20]:
print(service.swagger_uri)

http://a2b21d48-478e-463e-b747-72c9f3b14d27.southcentralus.azurecontainer.io/swagger.json


In [ ]:
scoring_uri = service.scoring_uri

data = {
  "data": [
    {
      "Age": 25,
      "SystolicBP":130 ,
      "DiastolicBP":80 ,
      "BS": 15,
      "BodyTemp": 98,
      "HeartRate": 86
          }
  ]
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

In [18]:
print(service.get_logs())

2022-03-21T17:52:14,871264100+00:00 - iot-server/run 
2022-03-21T17:52:14,872522500+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-03-21T17:52:14,880628500+00:00 - rsyslog/run 
2022-03-21T17:52:14,944990800+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-03-21T17:52:15,310600400+00:00 - iot-server/finish 1 0
2022-03-21T17:52:15,317438200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-03-21 17:52:20,172 | root | INFO | Starting up app insights client
logging socket was

In [21]:
service.delete()
model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
